In [12]:
#default and various
import pandas as pd
import numpy as np
import plotly.express as px
import os
import math

#datetime
from datetime import datetime
from datetime import timedelta

#ml-sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [13]:
g_dir : str         = "g_drive_data"
ip_dir : str        = "ip_forecast_data"
main_dir :str       = "main_data"

In [14]:
df_main_ip          = pd.read_csv(os.path.join(main_dir, "df_main_interpolated.csv"))

## 3. Feature engineering / adding
### 1. Timestamps transformation with sin() and cos()
Relevant feautre for engineering: utc_time
N of features added = 4
 - day
 - year
 
source: https://www.kaggle.com/code/franco12/timestamp-with-sin-and-cosine

In [15]:
#remove time zone sensitivity
df_main_ip["utc_time"] = pd.to_datetime(df_main_ip["utc_time"]).dt.tz_localize(None)

#timestamp conversion

day : int = 24 * 60 * 60

df_btc_cleaned["day_sin"] = np.sin(df_btc_cleaned["time"] * (2*np.pi / day))
df_eth_cleaned["day_sin"] = np.sin(df_eth_cleaned["time"] * (2*np.pi / day))

df_btc_cleaned["day_cos"] = np.cos(df_btc_cleaned["time"] * (2*np.pi / day))
df_eth_cleaned["day_cos"] = np.cos(df_eth_cleaned["time"] * (2*np.pi / day))


In [16]:
#conver dt object to timestamp for calcultaion

df_main_ip["utc_timestamp"] = df_main_ip["utc_time"].apply(datetime.timestamp)

#calculate sin and cos of day and year

day : int = 24 * 60 * 60 #[sec]
year : int = day * 366 #[sec] : 1,\n2020 was a leap year

df_main_ip["day_sin"] = np.sin(df_main_ip["utc_timestamp"] * (2*np.pi / day))
df_main_ip["day_cos"] = np.cos(df_main_ip["utc_timestamp"] * (2*np.pi / day))

df_main_ip["year_sin"] = np.sin(df_main_ip["utc_timestamp"] * (2*np.pi / year))
df_main_ip["year_cos"] = np.cos(df_main_ip["utc_timestamp"] * (2*np.pi / year))

In [17]:
fig_day = px.line(
    data_frame = df_main_ip.iloc[:10000:10,:],
    x = "utc_time",
    y = ["day_sin","day_cos"],
    title = "Day transformed"
)

fig_day.show()

fig_year = px.line(
    data_frame = df_main_ip.iloc[::1000,:],
    x = "utc_time",
    y = ["year_sin","year_cos"],
    title = "Year transformed"
)

fig_year.show()

### 3.2 Add positional data of turbines to scada data

In [18]:
#Front turbines: (30, 31, 32, 33)       coded value = 0
#Back: (26, 27, 28, 29, 34, 35)         coded value = 1

#setting up mapping dict for data frame
position_map : dict = {
    30 : "front",
    31 : "front",
    32 : "front",
    33 : "front",
    26 : "back",
    27 : "back",
    28 : "back",
    29 : "back",
    34 : "back",
    35 : "back",
}

#add inforamtion to df
df_main_ip["position"] = df_main_ip["wt_id"].map(position_map)

### 3.3 Months as a seperate feature (integer)

In [19]:
df_main_ip["month"] = df_main_ip["utc_time"].dt.month

In [20]:
# 3.4 does not seem relevant

In [21]:
#get the non interpolated data with the added features

#relevant_timestamps = pd.read_csv(os.path.join(g_dir, "forecasts_temp.csv"))
#data_points = pd.to_datetime(relevant_timestamps["init"]).dt.tz_localize(None)
#df_main_raw = df_main_ip[pd.to_datetime(df_main_ip["init_6"]).dt.tz_localize(None).isin(data_points)]

In [22]:
#save the pre processed data frames (fe = feature enginerred)

df_main_ip.to_csv(os.path.join(main_dir, "df_main_interpolated_fe.csv"), index=None)
#df_main_raw.to_csv(os.path.join(main_dir, "df_main_rawdata_fe.csv"), index=None)